In [1]:
import pandas as pd
import numpy as np

In [2]:
oscar_df = pd.read_csv('the_oscar_award.csv')
oscar_df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10390,2019,2020,92,WRITING (Original Screenplay),"Screenplay by Bong Joon Ho, Han Jin Won; Story...",Parasite,True
10391,2019,2020,92,JEAN HERSHOLT HUMANITARIAN AWARD,Geena Davis,NaN,True
10392,2019,2020,92,HONORARY AWARD,David Lynch,NaN,True
10393,2019,2020,92,HONORARY AWARD,Wes Studi,NaN,True


In [3]:
#reduce the years
oscar_current=oscar_df.loc[oscar_df["year_film"] > 1979, :]
#drop awards without films attached (lifetime achievement, etc)
oscar_current=oscar_current.dropna(subset=['film'])

#create new colum for nominees who are not winners
oscar_current["loser"]=oscar_current["winner"]!=True

oscar_current.head()

,year_film,year_ceremony,ceremony,category,name,film,winner,loser
5835,1980,1981,53,ACTOR IN A LEADING ROLE,Robert De Niro,Raging Bull,True,False
5836,1980,1981,53,ACTOR IN A LEADING ROLE,Robert Duvall,The Great Santini,False,True
5837,1980,1981,53,ACTOR IN A LEADING ROLE,John Hurt,The Elephant Man,False,True
5838,1980,1981,53,ACTOR IN A LEADING ROLE,Jack Lemmon,Tribute,False,True
5839,1980,1981,53,ACTOR IN A LEADING ROLE,Peter O'Toole,The Stunt Man,False,True


In [4]:
#groupby film and sum up winners and losers
winner_ct=oscar_current.groupby('film')['winner'].apply(sum)
loser_ct=oscar_current.groupby('film')['loser'].apply(sum)

count_df = pd.concat([winner_ct, loser_ct], axis=1)
count_df=count_df.rename(columns={'winner':'winner_ct', 
                                   'loser': 'loser_ct', 
                                  })



count_df

,winner_ct,loser_ct
film,,
'38',0,1
'Crocodile' Dundee,0,1
'Round Midnight,1,1
(A) Torzija [(A) Torsion],0,1
102 Dalmatians,0,1
...,...,...
Zus & Zo,0,1
"sex, lies, and videotape",0,1
the accountant,1,0


In [5]:
count_df['nominees']= count_df['winner_ct'] + count_df['loser_ct']
count_df.head()

,winner_ct,loser_ct,nominees
film,,,
'38',0,1,1
'Crocodile' Dundee,0,1,1
'Round Midnight,1,1,2
(A) Torzija [(A) Torsion],0,1,1
102 Dalmatians,0,1,1


In [6]:
oscar_ct=oscar_current.merge(count_df, on='film')
oscar_ct.head(10)

,year_film,year_ceremony,ceremony,category,name,film,winner,loser,winner_ct,loser_ct,nominees
0,1980,1981,53,ACTOR IN A LEADING ROLE,Robert De Niro,Raging Bull,True,False,2,6,8
1,1980,1981,53,ACTOR IN A SUPPORTING ROLE,Joe Pesci,Raging Bull,False,True,2,6,8
2,1980,1981,53,ACTRESS IN A SUPPORTING ROLE,Cathy Moriarty,Raging Bull,False,True,2,6,8
3,1980,1981,53,CINEMATOGRAPHY,Michael Chapman,Raging Bull,False,True,2,6,8
4,1980,1981,53,DIRECTING,Martin Scorsese,Raging Bull,False,True,2,6,8
5,1980,1981,53,FILM EDITING,Thelma Schoonmaker,Raging Bull,True,False,2,6,8
6,1980,1981,53,BEST PICTURE,"Irwin Winkler and Robert Chartoff, Producers",Raging Bull,False,True,2,6,8
7,1980,1981,53,SOUND,"Donald O. Mitchell, Bill Nicholson, David J. K...",Raging Bull,False,True,2,6,8
8,1980,1981,53,ACTOR IN A LEADING ROLE,Robert Duvall,The Great Santini,False,True,0,2,2
9,1980,1981,53,ACTOR IN A SUPPORTING ROLE,Michael O'Keefe,The Great Santini,False,True,0,2,2


In [7]:
oscar_ct.to_csv("the_oscar_award(version 2).csv", encoding="utf-8", index=False)